# <i>Create Disulfide Cross-Link</i>
### Given [1] a directory of pdb files and [2] two residue numbers:
- Mutate the residues to cysteines
- Create a bond between the sulfur atoms
- Minimize the structure
- Save the pdbs to a new "output" directory
- Add "SSBOND" line to pdb

In [1]:
import os
import subprocess

### Set directory and residue numbers

In [2]:
DIRECTORY = '/Users/alexandertaguchi/Desktop/slink_no_helix/'
RESIDUES = (1, 21)

### Generate chimera script

In [3]:
# Paths to pdbs and output directory
pdbs = [x for x in os.listdir(DIRECTORY) if x[-4:] == '.pdb']
if not os.path.isdir(DIRECTORY + 'output'):
    os.mkdir(DIRECTORY + 'output')

# Write command script to file
with open(DIRECTORY + 'ssbond.cmd', 'w') as file:
    file.write('#\n# Chimera command script to create disulfide bond\n#\n')
    
    # Loop over pdbs
    for pdb in pdbs:
        file.write(f'open {DIRECTORY + pdb};\n'
                   f'swapaa CYS #0:{RESIDUES[0]};\n'
                   f'bond #0:{RESIDUES[0]}@SG #0:{RESIDUES[1]}@SG\n'
                   f'minimize spec #0 nogui true\n'
                   f'~bond #0:{RESIDUES[0]}@SG #0:{RESIDUES[1]}@SG\n'
                   f'write #0 {DIRECTORY}output/{pdb[:-4]}_ss.pdb\n'
                   f'close #0\n')

### Run Chimera

In [4]:
subprocess.call(['chimera', '--nogui', DIRECTORY + 'ssbond.cmd'])

0

### Add SSBOND

In [7]:
for pdb in [x for x in os.listdir(DIRECTORY + 'output') if x[-7:] == '_ss.pdb']:

    # Retrieve pdb contents
    with open(DIRECTORY + 'output/' + pdb, 'r') as file:
        contents = file.readlines()
    
    # Insert SSBOND line
    index = [x.split()[0] for x in contents].index('ATOM')
    contents.insert(index, f'SSBOND CYS A {RESIDUES[0]} CYS A {RESIDUES[1]} 2.05\n')
    contents = "".join(contents)

    # Overwrite pdb file
    with open(DIRECTORY + 'output/' + pdb, 'w') as file:
        file.write(contents)